In [ ]:
# import Libraries
import random
import os
import gc
import random
random.seed(3)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
!pip install -q --upgrade fastcore 
!pip install -q --upgrade fastai # Make sure we have the latest fastai
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image as PImage # TO avoid confusion later
from fastai.vision.all import *


In [ ]:
# Getting Google Credentials and Authentication
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# unzipping files
!unzip "/content/gdrive/My Drive/CGIAR_Wheat_SampleSubmission.zip"
!unzip "/content/gdrive/My Drive/CGIAR_Wheat_Train_Dataset.zip"
!unzip "/content/gdrive/My Drive/Images.zip"

In [ ]:
# Use only high quality images 
train = pd.read_csv('Train.csv')
train.head(5)
train_hq = train[train['label_quality']== 2]
train_hq.head(2)

In [ ]:
#high quality data labels
train_hq.growth_stage.unique()

In [ ]:
# image transformation
 
# # Image Augmentation used for Densenet201 and  lr_max = slice (10e-7, 10e-6) with 40 epochs
# tfms = setup_aug_tfms([Brightness(max_lighting=0.9, p=1.,),
#                        CropPad(size=256),
#                        Zoom(max_zoom=1.5, p=0.5),
#                        Warp(magnitude=0.5, p=0.5)
#                       ])


# Image Augmentation used for Resnet50 and lr_max = slice (3e-6) with 40 epochs
tfms = setup_aug_tfms([Rotate(max_deg=15, p=1.),
            RandomResizedCrop(p=1., size=256),
            Brightness(max_lighting=1.0, p=0.4),
            CropPad(size=256),
            Zoom(max_zoom=1.0, p=0.2,
            Warp(magnitude=0.2, p=0.2)
            ])


# # Image Augmentation used for Vgg16_bn and lr_max = slice (10e-6) with 40 epochs
# tfms = setup_aug_tfms([Brightness(max_lighting=0.9, p=1.,),
#                        CropPad(size=256),
#                        Zoom(max_zoom=1.1, p=0.5),
#                        Warp(magnitude=0.2, p=0.5)
#                       ])



In [ ]:

# Create our dataloaders
dls = ImageDataLoaders.from_df(train_train, # train_hq if you only want the smaller high-quality set of images
                               path='Images/', 
                               suff='.jpeg', 
                               label_col='growth_stage', item_tfms=Resize(224), 
                               y_block=RegressionBlock ,
                               batch_tfms = tfms) #The only other change!
dls.show_batch()


In [ ]:
# Training the model
learn = cnn_learner(dls, resnet50, metrics=rmse, pretrained=True).to_fp16()
learn.fine_tune(70) 

In [ ]:
#training and validation loss
learn.recorder.plot_loss()

In [ ]:
#unfreeze last layer and find optimum learning rate
learn.unfreeze()
learn.lr_find()

In [ ]:
#train last layer only
learn.fit_one_cycle(40, lr_max=slice(10e-7, 10e-5))

In [ ]:
#prediction for submission file
# Making test dataloader and Test Time Augementation
ss = pd.read_csv('SampleSubmission.csv')
tdl = learn.dls.test_dl(ss) # Make test dataloader
preds = learn.tta(dl=tdl) # Test Time Augmentation